In [41]:
from dotenv import load_dotenv
import os
# Carga las variables del archivo .env
load_dotenv()

True

In [ ]:
from IPython.display import display, HTML
import ipywidgets as widgets

# Bloque HTML para el título con estilos personalizados
header_html = """
<div style="background-color:#EB001F; padding:20px; text-align:center; margin-bottom:20px;">
  <h1 style="color:white; margin:0;">DaviTravel - Davivienda</h1>
  <h3 style="color:white; margin-top:5px;">Su viaje puede estar en el lugar equivocado, planéelo con DaviTravel</h3>
  <p style="font-size:14px; color:white; margin-top:10px;">
    Complete la siguiente información para diseñar el plan de viaje perfecto.
  </p>
</div>
"""
display(HTML(header_html))

# Función auxiliar para asignar "Indiferente" si el campo está vacío
def valor_o_indiferente(valor):
    return valor if valor.strip() != "" else "Indiferente"

# Layout base para los widgets
layout_text = widgets.Layout(width='600px')
layout_area = widgets.Layout(width='600px', height='100px')

# Campos del formulario
origen = widgets.Text(
    value='',
    placeholder='Ej: "Bogotá" o "Bogotá, Colombia"',
    description='¿Desde dónde viaja?',
    style={'description_width': '200px'},
    layout=layout_text
)
origen_explicacion = widgets.HTML(
    value='<i>Describa el punto de partida del viaje. Ej: "Bogotá, Colombia".</i>'
)

destino = widgets.Text(
    value='',
    placeholder='Ej: "Cartagena" o "Torre Eiffel, París, Francia"',
    description='¿Hacia dónde se dirige?',
    style={'description_width': '200px'},
    layout=layout_text
)
destino_explicacion = widgets.HTML(
    value='<i>Mencione el destino o destinos deseados. Ej: "Cartagena" o "Torre Eiffel, París".</i>'
)

fecha = widgets.Text(
    value='',
    placeholder='Ej: "a mediados de julio", "en agosto" o "el próximo año"',
    description='¿Cuándo viajar?',
    style={'description_width': '200px'},
    layout=layout_text
)
fecha_explicacion = widgets.HTML(
    value='<i>Indique el periodo en el que se planea el viaje, ya sea con fechas específicas o términos generales.</i>'
)

duracion = widgets.Text(
    value='',
    placeholder='Ej: "entre 2 semanas y un mes" o "10 días"',
    description='Duración del viaje:',
    style={'description_width': '200px'},
    layout=layout_text
)
duracion_explicacion = widgets.HTML(
    value='<i>Especifique la duración del viaje, por ejemplo, "entre 2 semanas y un mes".</i>'
)

tipo_viaje = widgets.Text(
    value='',
    placeholder='Ej: "un viaje cultural con descanso", "una aventura urbana", "un recorrido de negocios innovador" o "mixto"',
    description='Tipo de viaje:',
    style={'description_width': '200px'},
    layout=layout_text
)
tipo_viaje_explicacion = widgets.HTML(
    value='<i>Describa el enfoque del viaje. Por ejemplo, "un viaje cultural con descanso" o "una aventura urbana".</i>'
)

perfil_viajero = widgets.Text(
    value='',
    placeholder='Ej: "viaja solo", "con familia", "con amigos", "con pareja y compañeros de trabajo", etc.',
    description='¿Con quién viaja?',
    style={'description_width': '200px'},
    layout=layout_text
)
perfil_viajero_explicacion = widgets.HTML(
    value='<i>Indique quiénes acompañan al viajero. Por ejemplo, "viaja solo" o "con familia y amigos".</i>'
)

presupuesto = widgets.Text(
    value='',
    placeholder='Ej: "alrededor de 1500000 COP"',
    description='Presupuesto (COP):',
    style={'description_width': '200px'},
    layout=layout_text
)
presupuesto_explicacion = widgets.HTML(
    value='<i>Escriba el presupuesto en pesos colombianos. Si se deja vacío, se interpretará como "Indiferente".</i>'
)

expectativas = widgets.Textarea(
    value='',
    placeholder='Ej: "Desea combinar visitas a lugares emblemáticos con experiencias auténticas y gastronomía local"',
    description='Expectativas adicionales:',
    style={'description_width': '200px'},
    layout=layout_area
)
expectativas_explicacion = widgets.HTML(
    value='<i>Comparta cualquier detalle o preferencia adicional respecto al viaje.</i>'
)

# Botón para enviar la información
submit_button = widgets.Button(
    description='Enviar Información',
    button_style='success'
)

output = widgets.Output()

def on_submit_clicked(b):
    with output:
        output.clear_output()
        print("Información del Viaje:")
        print("Desde dónde viaja:", valor_o_indiferente(origen.value))
        print("Hacia dónde se dirige:", valor_o_indiferente(destino.value))
        print("¿Cuándo viajar:", valor_o_indiferente(fecha.value))
        print("Duración del viaje:", valor_o_indiferente(duracion.value))
        print("Tipo de viaje:", valor_o_indiferente(tipo_viaje.value))
        print("Con quién viaja:", valor_o_indiferente(perfil_viajero.value))
        print("Presupuesto (COP):", valor_o_indiferente(presupuesto.value))
        print("Expectativas adicionales:", valor_o_indiferente(expectativas.value))

submit_button.on_click(on_submit_clicked)

# Organiza el formulario en un contenedor blanco interno
form_inner = widgets.VBox([
    widgets.VBox([origen, origen_explicacion]),
    widgets.VBox([destino, destino_explicacion]),
    widgets.VBox([fecha, fecha_explicacion]),
    widgets.VBox([duracion, duracion_explicacion]),
    widgets.VBox([tipo_viaje, tipo_viaje_explicacion]),
    widgets.VBox([perfil_viajero, perfil_viajero_explicacion]),
    widgets.VBox([presupuesto, presupuesto_explicacion]),
    widgets.VBox([expectativas, expectativas_explicacion]),
    submit_button,
    output
], layout=widgets.Layout(
    border='2px solid #fff',
    background_color='#ffffff',
    padding='20px'
))

# Organiza el formulario completo en un contenedor externo (fondo rojo)
form_outer = widgets.VBox([form_inner], layout=widgets.Layout(
    background_color='#ed1c27',
    padding='20px',
    width='100%',
    align_items='center'
))

display(form_outer)

def construir_consulta() -> str:
    """
    Construye la cadena de consulta a partir de los valores ingresados en el formulario.
    Si algún campo está vacío, se considera "Indiferente".
    """
    consulta = (
        "Desde dónde viaja: " + valor_o_indiferente(origen.value) + "; " +
        "Hacia dónde se dirige: " + valor_o_indiferente(destino.value) + "; " +
        "Cuándo viajar: " + valor_o_indiferente(fecha.value) + "; " +
        "Duración del viaje: " + valor_o_indiferente(duracion.value) + "; " +
        "Tipo de viaje: " + valor_o_indiferente(tipo_viaje.value) + "; " +
        "Con quién viaja: " + valor_o_indiferente(perfil_viajero.value) + "; " +
        "Presupuesto: " + valor_o_indiferente(presupuesto.value) + "; " +
        "Expectativas adicionales: " + valor_o_indiferente(expectativas.value)
    )
    return consulta

In [43]:
import time
from collections import deque
import os
import requests

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import GoogleSearchAPIWrapper

In [44]:
# --------------------------------------------------
# 1. RateLimiter para 15 llamadas/minuto
# --------------------------------------------------
class RateLimiter:
    def __init__(self, max_calls: int, period: float):
        self.max_calls = max_calls
        self.period = period
        self.calls = deque()
    def wait(self):
        now = time.monotonic()
        while self.calls and now - self.calls[0] > self.period:
            self.calls.popleft()
        if len(self.calls) >= self.max_calls:
            to_wait = self.period - (now - self.calls[0])
            time.sleep(to_wait)
            now = time.monotonic()
            while self.calls and now - self.calls[0] > self.period:
                self.calls.popleft()
        self.calls.append(now)

limiter = RateLimiter(max_calls=15, period=60.0)

In [45]:
# --------------------------------------------------
# 2. Instanciar el LLM “vanilla” de Google Gemini
# --------------------------------------------------
gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=os.environ["GOOGLE_API_KEY"],
    temperature=0.3
)

In [46]:
# ========================================
# 🔍 EXTRACCIÓN DE DATOS DEL FORMULARIO
# ========================================

prompt_extraccion = PromptTemplate(
    input_variables=["datos_formulario"],
    template=(
        "Extraiga los siguientes datos del texto: origen, destino, fecha, duración del viaje, tipo de viaje, acompañantes, presupuesto y expectativas.\n\n"
        "Texto:\n{datos_formulario}\n\nResponda en formato JSON corto."
    )
)
chain_extraccion = LLMChain(llm=gemini_llm, prompt=prompt_extraccion)

# ========================================
# 🌍 DIMENSIÓN MIGRATORIA
# ========================================

prompt_migratoria = PromptTemplate(
    input_variables=["info_requerimientos"],
    template=(
        "Dado este requerimiento de viaje: {info_requerimientos}, "
        "genere una sola consulta de búsqueda en español (máximo una línea) para buscar los requisitos migratorios en internet."
    )
)
chain_migratoria = LLMChain(llm=gemini_llm, prompt=prompt_migratoria)

prompt_consulta_migratoria = PromptTemplate(
    input_variables=["contexto"],
    template=(
        "Con base en este contexto migratorio generado por el sistema:\n\n{contexto}\n\n"
        "Escriba una consulta de búsqueda breve y específica en español para internet. "
        "La consulta debe buscar requisitos migratorios oficiales para ciudadanos colombianos hacia el país destino. "
        "No escriba explicaciones, solo la consulta en una línea."
    )
)

prompt_migratoria_final = PromptTemplate(
    input_variables=["info_requerimientos"],
    template=(
        "Con base en esta información y los resultados web, indique los requisitos migratorios para el viaje:\n\n"
        "{info_requerimientos}\n\n"
        "Sea claro y conciso. Liste si se requiere visa, vacunas u otros documentos."
    )
)
chain_migratoria_final = LLMChain(llm=gemini_llm, prompt=prompt_migratoria_final)

# ========================================
# 🏖️ DIMENSIÓN TURISMO
# ========================================

prompt_turismo = PromptTemplate(
    input_variables=["info_requerimientos"],
    template=(
        "Describa brevemente el clima, las temporadas turísticas (alta o baja) y eventos relevantes en el destino según esta información:\n\n"
        "{info_requerimientos}\n\nUse viñetas si es posible. Sea claro y evite detalles innecesarios."
    )
)
chain_turismo = LLMChain(llm=gemini_llm, prompt=prompt_turismo)

prompt_consulta_turismo = PromptTemplate(
    input_variables=["contexto"],
    template=(
        "Según este contexto turístico:\n\n{contexto}\n\n"
        "Genere una consulta de búsqueda concreta y corta en español para internet. "
        "Debe enfocarse en eventos turísticos, clima o temporadas en el destino en las fechas del viaje. "
        "No incluya explicaciones, solo la consulta."
    )
)

prompt_turismo_final = PromptTemplate(
    input_variables=["info_requerimientos"],
    template=(
        "Utilice la siguiente información y resultados de búsqueda para resumir el clima, eventos y temporada turística del destino:\n\n"
        "{info_requerimientos}\n\nUse viñetas si es posible. Sea concreto."
    )
)
chain_turismo_final = LLMChain(llm=gemini_llm, prompt=prompt_turismo_final)

# ========================================
# 💸 DIMENSIÓN COSTOS
# ========================================

prompt_costos = PromptTemplate(
    input_variables=["info_requerimientos"],
    template=(
        "Con base en la información del viaje, estime el costo diario en COP de:\n- Transporte\n- Alojamiento\n- Comida\n\n"
        "Datos:\n{info_requerimientos}\n\nPresente el valor aproximado por cada concepto."
    )
)
chain_costos = LLMChain(llm=gemini_llm, prompt=prompt_costos)

prompt_consulta_costos = PromptTemplate(
    input_variables=["contexto"],
    template=(
        "Este es un resumen de contexto sobre los costos estimados del viaje:\n\n{contexto}\n\n"
        "Cree una consulta en español (una línea) para buscar en internet información sobre precios promedio en transporte, alojamiento o comida "
        "en el destino turístico. Enfóquese en el país y ciudad destino, si se conoce. "
        "No escriba justificaciones, solo la consulta."
    )
)

prompt_costos_final = PromptTemplate(
    input_variables=["info_requerimientos"],
    template=(
        "Con base en la información siguiente, entregue una estimación de costos diarios en COP para:\n"
        "- Transporte\n- Alojamiento\n- Alimentación\n\n"
        "{info_requerimientos}\n\nPresente solo los valores aproximados por concepto."
    )
)
chain_costos_final = LLMChain(llm=gemini_llm, prompt=prompt_costos_final)

# ========================================
# 📊 DIMENSIÓN PRESUPUESTO
# ========================================

prompt_presupuesto = PromptTemplate(
    input_variables=["costos"],
    template=(
        "Con los siguientes costos estimados, calcule el total y compare con el presupuesto indicado por el usuario. "
        "Si excede, sugiera 1 o 2 formas de reducir gastos:\n\n{costos}"
    )
)
chain_presupuesto = LLMChain(llm=gemini_llm, prompt=prompt_presupuesto)

# ========================================
# 🧳 ITINERARIO FINAL
# ========================================

prompt_itinerario = PromptTemplate(
    input_variables=["info_migratoria", "info_turismo", "presupuesto_final", "info_usuario"],
    template=(
        "Con base en la siguiente información, genere un itinerario de viaje detallado pero conciso:\n\n"
        "0. Preferencias del usuario:\n{info_usuario}\n\n"
        "1. Requisitos migratorios: {info_migratoria}\n"
        "2. Información turística: {info_turismo}\n"
        "3. Presupuesto y costos estimados: {presupuesto_final}\n\n"
        "Presente el resultado en las siguientes secciones:\n\n"
        "**I. Documentos y requisitos:**\n"
        "- Enumere los documentos necesarios para el ingreso y salida del país.\n"
        "- Indique si se necesita visa, vacunas, o formularios especiales.\n\n"
        "**II. Información general del destino:**\n"
        "- Clima estimado en las fechas del viaje.\n"
        "- Temporada (alta o baja).\n"
        "- Eventos especiales que coincidan con el viaje.\n\n"
        "**III. Presupuesto estimado:**\n"
        "- Desglose en COP por transporte, alojamiento y alimentación.\n"
        "- Nota si se ajusta al presupuesto definido o si se requieren cambios.\n\n"
        "**IV. Itinerario diario sugerido:**\n"
        "Presente de forma breve un plan por día. Para cada día, indique:\n"
        "- Actividades principales (máximo 3 por día).\n"
        "- Tipo de actividad (cultural, gastronómica, naturaleza, descanso, etc.).\n"
        "- Sugerencias de horario o ubicación si aplica.\n\n"
        "**V. Recomendaciones finales:**\n"
        "- Consejos adicionales según el perfil del viaje (por ejemplo, si es romántico, familiar, de aventura, etc.).\n"
        "- Recomendaciones sobre transporte local, seguridad o reservas anticipadas.\n\n"
        "El texto debe ser claro, útil y concreto. Use listas o viñetas cuando sea posible para facilitar la lectura."
    )
)
chain_itinerario = LLMChain(llm=gemini_llm, prompt=prompt_itinerario)

In [47]:
from duckduckgo_search import DDGS

def enriquecer_con_internet(
    prompt_contextual: PromptTemplate, chain_contextual: LLMChain,
    prompt_consulta: PromptTemplate,
    prompt_final: PromptTemplate, chain_final: LLMChain,
    info_usuario: str, nombre_cadena: str
) -> str:
    """
    Genera contexto con el LLM, realiza búsqueda en internet y ejecuta una cadena final
    enriquecida, registrando logs detallados en cada paso.
    """
    # Paso 1: Generar contexto base con el modelo
    prompt_ctx = prompt_contextual.format(info_requerimientos=info_usuario)
    limiter.wait()
    contexto_base = chain_contextual.run({"info_requerimientos": info_usuario})
    log_llm(f"{nombre_cadena} - Contexto Base", prompt_ctx, contexto_base)

    # Paso 2: Generar la consulta de búsqueda usando el contexto generado
    consulta_prompt = prompt_consulta.format(contexto=contexto_base)
    limiter.wait()
    consulta_generada = gemini_llm.invoke(consulta_prompt).content.strip()
    consulta_simple = consulta_generada.replace("`", "").split("\n")[0].strip()
    log_llm(f"{nombre_cadena} - Consulta Web", consulta_prompt, consulta_generada)

    # Paso 3: Ejecutar búsqueda en internet
    limiter.wait()
    resultados = []
    with DDGS() as ddgs:
        for r in ddgs.text(consulta_simple, max_results=5):
            resultados.append(f"{r['title']}: {r['href']}")
    contexto_internet = "\n".join(resultados)

    # Guardar los resultados de búsqueda como parte del contexto
    log_llm(f"{nombre_cadena} - Resultados Web", consulta_simple, contexto_internet)

    # Paso 4: Crear contexto combinado
    contexto_completo = f"{contexto_base}\n\nResultados de internet:\n{contexto_internet}"

    # Paso 5: Ejecutar la cadena final con el contexto enriquecido
    prompt_final_text = prompt_final.format(info_requerimientos=contexto_completo)
    limiter.wait()
    resultado = chain_final.run({"info_requerimientos": contexto_completo})
    log_llm(f"{nombre_cadena} - Resultado Final", prompt_final_text, resultado)

    return resultado

In [48]:
# --------------------------------------------------
# 5. Función para Generar el Plan de Viaje con Rate Limiting y Logs
# --------------------------------------------------
def generar_plan_de_viaje(info_requerimientos: str) -> str:
    limiter.wait()
    migratoria = enriquecer_con_internet(
        prompt_contextual=prompt_migratoria,
        chain_contextual=chain_migratoria,
        prompt_consulta=prompt_consulta_migratoria,
        prompt_final=prompt_migratoria_final,
        chain_final=chain_migratoria_final,
        info_usuario=info_requerimientos,
        nombre_cadena="Información Migratoria"
    )

    limiter.wait()
    turismo = enriquecer_con_internet(
        prompt_contextual=prompt_turismo,
        chain_contextual=chain_turismo,
        prompt_consulta=prompt_consulta_turismo,
        prompt_final=prompt_turismo_final,
        chain_final=chain_turismo_final,
        info_usuario=info_requerimientos,
        nombre_cadena="Información Turística"
    )

    limiter.wait()
    costos = enriquecer_con_internet(
        prompt_contextual=prompt_costos,
        chain_contextual=chain_costos,
        prompt_consulta=prompt_consulta_costos,
        prompt_final=prompt_costos_final,
        chain_final=chain_costos_final,
        info_usuario=info_requerimientos,
        nombre_cadena="Estimación de Costos"
    )

    limiter.wait()
    presupuesto_final = chain_presupuesto.run({"costos": costos})
    log_llm("Cálculo de Presupuesto", prompt_presupuesto.format(costos=costos), presupuesto_final)

    limiter.wait()
    itinerario = chain_itinerario.run({
    "info_migratoria": migratoria,
    "info_turismo": turismo,
    "presupuesto_final": presupuesto_final,
    "info_usuario": info_requerimientos
    })

    log_llm(
        "Itinerario Final",
        prompt_itinerario.format(
            info_migratoria=migratoria,
            info_turismo=turismo,
            presupuesto_final=presupuesto_final,
            info_usuario=info_requerimientos
        ),
        itinerario
    )


    return itinerario

In [49]:
# --------------------------------------------------
# 6. Definición de la Herramienta Personalizada TravelPlannerTool
# --------------------------------------------------
class TravelPlannerTool(BaseTool):
    name: str = "TravelPlannerTool"
    description: str = "Planificador de viajes: recibe una consulta y retorna un resumen completo del plan de viaje."
    def _run(self, query: str) -> str:
        limiter.wait()
        info = chain_extraccion.run({"datos_formulario": query})
        log_llm("Extracción de Requerimientos", prompt_extraccion.format(datos_formulario=query), info)
        return generar_plan_de_viaje(info)
    async def _arun(self, query: str) -> str:
        raise NotImplementedError


In [50]:

# --------------------------------------------------
# 7. Inicialización del Agente con la Herramienta Personalizada
# --------------------------------------------------
from langchain.agents import initialize_agent

travel_planner_tool = TravelPlannerTool()
agent = initialize_agent(
    tools=[travel_planner_tool],
    llm=gemini_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    return_intermediate_steps=True
)

In [52]:
# --------------------------------------------------
# 8. Integración con el Formulario y Ejecución Final
# --------------------------------------------------
from datetime import datetime

llm_log = []

def log_llm(nombre_cadena: str, prompt_text: str, respuesta: str):
    llm_log.append({
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "nombre_cadena": nombre_cadena,
        "prompt": prompt_text,
        "respuesta": respuesta
    })

import pandas as pd

def exportar_llm_log_a_excel(nombre_archivo="llm_log.xlsx"):
    if not llm_log:
        print("No hay registros para exportar.")
        return
    
    df = pd.DataFrame(llm_log)
    df = df.sort_values("timestamp")  # Asegura orden cronológico
    df.to_excel(nombre_archivo, index=False)
    print(f"✅ Log exportado exitosamente a: {nombre_archivo}")

consulta_agente = construir_consulta()
# Ejecutar el agente capturando pasos intermedios
limiter.wait()
result = agent.invoke({"input": consulta_agente}, return_intermediate_steps=True)
exportar_llm_log_a_excel()

# Extraer la última Observation (que contiene el resultado del itinerario final)
ultimo_observation = result["intermediate_steps"][-1][1]  # [1] es el observation

# Mostrar solo el itinerario generado
print("\n" + "=" * 60)
print("📦 ITINERARIO DETALLADO DEL VIAJE")
print("=" * 60)
print(ultimo_observation.strip())

✅ Log exportado exitosamente a: llm_log.xlsx

📦 ITINERARIO DETALLADO DEL VIAJE
Aquí tienes un itinerario detallado y conciso para tu viaje de turismo ecológico a Costa Rica, basado en la información proporcionada y asumiendo una duración de **7 días**, **2 personas** viajando, y un presupuesto de **9.000.000 COP**.

**I. Documentos y requisitos:**

*   **Pasaporte:** Vigente con al menos seis meses de validez al momento del ingreso.
*   **Tiquete de salida:** Billete de avión de salida de Costa Rica (ida y vuelta).
*   **Prueba de solvencia económica:** Demostrar que se cuenta con los recursos económicos suficientes para cubrir los gastos durante la estadía (extractos bancarios, tarjetas de crédito).
*   **Visa:** Verificar si se necesita visa según la nacionalidad. Consultar la Embajada de Costa Rica en Colombia.
*   **Vacunas:** No se exigen vacunas obligatorias.
*   **Otros documentos:** Copia de la reserva del hotel o lugar de hospedaje, itinerario del viaje.

**II. Información gen